In [47]:
import pandas as pd
from os import listdir
from os.path import join
import pickle
from os import mkdir

In [48]:
# input_path = "../output/dateappended"
# output_path = "../output/adv_risk_affected_appended"

In [49]:
input_path = "../data/dateAppended"
output_path = "../data/final_adv_appended"

In [50]:
# mkdir(output_path)

In [51]:
adv_df = pd.read_csv("../data/reported_events.txt", sep="|")

In [52]:
print(adv_df.shape)

(4456107, 17)


In [53]:
adv_df.head()

,id,nct_id,result_group_id,ctgov_group_code,time_frame,event_type,default_vocab,default_assessment,subjects_affected,subjects_at_risk,description,event_count,organ_system,adverse_event_term,frequency_threshold,vocab,assessment
0,22938255,NCT03704376,4369989,E2,6 months,serious,NaN,NaN,0.0,40.0,NaN,NaN,Total,"Total, serious adverse events",0,NaN,NaN
1,22938256,NCT03704376,4369990,E1,6 months,serious,NaN,NaN,0.0,38.0,NaN,NaN,Total,"Total, serious adverse events",0,NaN,NaN
2,22938257,NCT03704376,4369989,E2,6 months,serious,NaN,NaN,0.0,40.0,NaN,NaN,Total,"Total, all-cause mortality",0,NaN,NaN
3,22938258,NCT03704376,4369990,E1,6 months,serious,NaN,NaN,0.0,38.0,NaN,NaN,Total,"Total, all-cause mortality",0,NaN,NaN
4,22938259,NCT03704376,4369989,E2,6 months,other,NaN,Non-systematic Assessment,1.0,40.0,NaN,1.0,Vascular disorders,Deep Vein Thrombosis,0,NaN,NaN


In [54]:
adv_df = adv_df.drop(['result_group_id', 'ctgov_group_code', 'time_frame', 'event_type', 'default_vocab', 'default_assessment', 'description', 'event_count', 'organ_system', 'adverse_event_term', 'frequency_threshold', 'vocab', 'assessment'], axis=1)

In [55]:
groups = adv_df.groupby('nct_id')

In [56]:
adv_dict = {}
for a, b in groups:
#     print(a)
#     print(b)
    tot_subj_affect = 0
    tot_subj_risk = 0
    
    for subj_aff in b.subjects_affected:
        tot_subj_affect += subj_aff
#         print(subj_aff)
    
    for subj_risk in b.subjects_at_risk:
        if subj_risk > 0:
            tot_subj_risk += subj_risk
#         print(subj_risk)
    newDict = {}
    newDict['tot_subj_affected'] = tot_subj_affect
    newDict['tot_subj_risk'] = tot_subj_risk
    adv_dict[a] = newDict
#     print(adv_dict)

In [57]:
pickle.dump(adv_dict, open("../data/adversialSubjRiskAndAffectedCount.p", "wb"))

In [58]:
adv_dict_count = pickle.load(open("../data/adversialSubjRiskAndAffectedCount.p", "rb"))

In [59]:
# mkdir("../output/adv_risk_affected_appended")

In [60]:
files = sorted(listdir(input_path))

In [61]:
files

['1_Pathological Conditions, Signs and Symptoms_12826_1_queryResultsOurMethod_pageRank_date_appendedadverse_appended_popularity_appended.csv',
 '1_Pathological Conditions, Signs and Symptoms_12826_2_queryResultsOurMethod_pageRank_date_appendedadverse_appended_popularity_appended.csv',
 '1_Pathological Conditions, Signs and Symptoms_12826_3_queryResultsOurMethod_pageRank_date_appendedadverse_appended_popularity_appended.csv',
 '1_Pathological Conditions, Signs and Symptoms_12826_4_queryResultsOurMethod_pageRank_date_appendedadverse_appended_popularity_appended.csv',
 '1_Pathological Conditions, Signs and Symptoms_12826_5_queryResultsOurMethod_pageRank_date_appendedadverse_appended_popularity_appended.csv',
 '2_Neoplasms_7398_1_queryResultsOurMethod_pageRank_date_appendedadverse_appended_popularity_appended.csv',
 '2_Neoplasms_7398_2_queryResultsOurMethod_pageRank_date_appendedadverse_appended_popularity_appended.csv',
 '2_Neoplasms_7398_3_queryResultsOurMethod_pageRank_date_appendedadve

In [62]:
for file in files:
    subj_aff_list = []
    subj_at_risk_list = []
    print(file)
    rel_df = pd.read_csv(join(input_path, file))
    for nct_id in rel_df.nct_id:
#         print(nct_id)
        if nct_id in adv_dict_count.keys():
            subj_aff_list.append(adv_dict_count[nct_id]['tot_subj_affected'])
            subj_at_risk_list.append(adv_dict_count[nct_id]['tot_subj_risk'])
        else:
            subj_aff_list.append("NA")
            subj_at_risk_list.append("NA")
    
    rel_df['tot_subj_affected'] = subj_aff_list
    rel_df['tot_subj_risk'] = subj_at_risk_list
    rel_df.to_csv(join(output_path, file.split(".")[0] + "adverse_appended" + ".csv"), index=False)    

1_Pathological Conditions, Signs and Symptoms_12826_1_queryResultsOurMethod_pageRank_date_appendedadverse_appended_popularity_appended.csv
1_Pathological Conditions, Signs and Symptoms_12826_2_queryResultsOurMethod_pageRank_date_appendedadverse_appended_popularity_appended.csv
1_Pathological Conditions, Signs and Symptoms_12826_3_queryResultsOurMethod_pageRank_date_appendedadverse_appended_popularity_appended.csv
1_Pathological Conditions, Signs and Symptoms_12826_4_queryResultsOurMethod_pageRank_date_appendedadverse_appended_popularity_appended.csv
1_Pathological Conditions, Signs and Symptoms_12826_5_queryResultsOurMethod_pageRank_date_appendedadverse_appended_popularity_appended.csv
2_Neoplasms_7398_1_queryResultsOurMethod_pageRank_date_appendedadverse_appended_popularity_appended.csv
2_Neoplasms_7398_2_queryResultsOurMethod_pageRank_date_appendedadverse_appended_popularity_appended.csv
2_Neoplasms_7398_3_queryResultsOurMethod_pageRank_date_appendedadverse_appended_popularity_append